In [10]:
import qiskit
from qiskit_optimization import QuadraticProgram
from qiskit_algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.optimizers import COBYLA
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as SamplerV2
from qiskit.primitives import Sampler
from qiskit_optimization.converters import QuadraticProgramToQubo
import qiskit_aer as Aer
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.converters import InequalityToEquality

In [11]:
Nc= 3 # Nc is the number of seconds

Dist=1 # Distance to travel

tolerance = 1 # Tolerance in distance travelled

delta_v=1 # Rate of acceleration/deceleration set to 1

vmax=1 # Max speed of a TGV in France (in m/s)

alpha=0.05 # Regenerative braking efficiency

qp = QuadraticProgram()

x={}
for i in range(0,Nc):
    qp.binary_var(f"x_{i}")
    qp.binary_var(f"y_{i}")
    qp.binary_var(f"z_{i}")

linear_dict = {}
for i in range(Nc):
    linear_dict[f"x_{i}"] = delta_v**2
    linear_dict[f"y_{i}"] = - alpha*delta_v**2

qp.minimize(linear=linear_dict)



qp.get_num_vars()
qp.prettyprint()

'Problem name: \n\nMinimize\n  x_0 + x_1 + x_2 - 0.05*y_0 - 0.05*y_1 - 0.05*y_2\n\nSubject to\n  No constraints\n\n  Binary variables (9)\n    x_0 y_0 z_0 x_1 y_1 z_1 x_2 y_2 z_2\n'

In [12]:
# Constraint 1: No simultaneous braking/acceleration
for i in range(0, Nc):
    qp.linear_constraint(linear={f'z_{i}': 1, f'x_{i}': -1}, 
                        sense='<=', 
                        rhs=0, 
                        name=f'z_u_d_{i}')

for i in range(0, Nc):
    qp.linear_constraint(linear={f'z_{i}': 1, f'y_{i}': -1}, 
                        sense='<=', 
                        rhs=0, 
                        name=f'z_p_d_{i}')

for i in range(0, Nc):
    qp.linear_constraint(linear={f'z_{i}': 1, f'x_{i}': -1, f'y_{i}': -1}, 
                        sense='>=', 
                        rhs=-1, 
                        name=f'z_u_p_d_{i}')

# Sum of z[i] should be 0
linear_sum = {f'z_{i}': 1 for i in range(Nc)}
qp.linear_constraint(linear=linear_sum,
                    sense='==',
                    rhs=0,
                    name='No_simultaneous_braking_or_acceleration_constraint')

# Print information about the model
qp.get_num_vars()

9

In [13]:
# Constraint 2: Total Distance constraints

# Add distance constraints with tolerance
qp.linear_constraint(linear={f'x_{i}': coef for i, coef in enumerate([delta_v*(Nc-i) for i in range(Nc)]) if coef != 0} | 
                    {f'y_{i}': -coef for i, coef in enumerate([delta_v*(Nc-i) for i in range(Nc)]) if coef != 0},
                    sense='<=',
                    rhs=Dist + tolerance,
                    name='Max_Distance_constraint')

qp.linear_constraint(linear={f'x_{i}': coef for i, coef in enumerate([delta_v*(Nc-i) for i in range(Nc)]) if coef != 0} | 
                    {f'y_{i}': -coef for i, coef in enumerate([delta_v*(Nc-i) for i in range(Nc)]) if coef != 0},
                    sense='>=',
                    rhs=Dist - tolerance,
                    name='Min_Distance_constraint')

# Print model information
qp.prettyprint()

"Problem name: \n\nMinimize\n  x_0 + x_1 + x_2 - 0.05*y_0 - 0.05*y_1 - 0.05*y_2\n\nSubject to\n  Linear constraints (12)\n    -x_0 + z_0 <= 0  'z_u_d_0'\n    -x_1 + z_1 <= 0  'z_u_d_1'\n    -x_2 + z_2 <= 0  'z_u_d_2'\n    -y_0 + z_0 <= 0  'z_p_d_0'\n    -y_1 + z_1 <= 0  'z_p_d_1'\n    -y_2 + z_2 <= 0  'z_p_d_2'\n    -x_0 - y_0 + z_0 >= -1  'z_u_p_d_0'\n    -x_1 - y_1 + z_1 >= -1  'z_u_p_d_1'\n    -x_2 - y_2 + z_2 >= -1  'z_u_p_d_2'\n    z_0 + z_1 + z_2 == 0  'No_simultaneous_braking_or_acceleration_constraint'\n    3*x_0 + 2*x_1 + x_2 - 3*y_0 - 2*y_1 - y_2 <= 2  'Max_Distance_constraint'\n    3*x_0 + 2*x_1 + x_2 - 3*y_0 - 2*y_1 - y_2 >= 0  'Min_Distance_constraint'\n\n  Binary variables (9)\n    x_0 y_0 z_0 x_1 y_1 z_1 x_2 y_2 z_2\n"

In [14]:
# Constraint 3: Net-Zero constraint (sum of acceleration equals sum of deceleration)
qp.linear_constraint(linear={f'x_{i}': -1 for i in range(Nc)} | {f'y_{i}': 1 for i in range(Nc)},
                    sense='==',
                    rhs=0,
                    name='Net_Zero_constraint')

# Print the optimization model info
qp.prettyprint()

"Problem name: \n\nMinimize\n  x_0 + x_1 + x_2 - 0.05*y_0 - 0.05*y_1 - 0.05*y_2\n\nSubject to\n  Linear constraints (13)\n    -x_0 + z_0 <= 0  'z_u_d_0'\n    -x_1 + z_1 <= 0  'z_u_d_1'\n    -x_2 + z_2 <= 0  'z_u_d_2'\n    -y_0 + z_0 <= 0  'z_p_d_0'\n    -y_1 + z_1 <= 0  'z_p_d_1'\n    -y_2 + z_2 <= 0  'z_p_d_2'\n    -x_0 - y_0 + z_0 >= -1  'z_u_p_d_0'\n    -x_1 - y_1 + z_1 >= -1  'z_u_p_d_1'\n    -x_2 - y_2 + z_2 >= -1  'z_u_p_d_2'\n    z_0 + z_1 + z_2 == 0  'No_simultaneous_braking_or_acceleration_constraint'\n    3*x_0 + 2*x_1 + x_2 - 3*y_0 - 2*y_1 - y_2 <= 2  'Max_Distance_constraint'\n    3*x_0 + 2*x_1 + x_2 - 3*y_0 - 2*y_1 - y_2 >= 0  'Min_Distance_constraint'\n    -x_0 - x_1 - x_2 + y_0 + y_1 + y_2 == 0  'Net_Zero_constraint'\n\n  Binary variables (9)\n    x_0 y_0 z_0 x_1 y_1 z_1 x_2 y_2 z_2\n"

In [15]:
# Constraint 4: Maximum Speed constraint
qp.linear_constraint(linear={f'x_{i}': delta_v for i in range(Nc)} | {f'y_{i}': -delta_v for i in range(Nc)},
                    sense='<=',
                    rhs=vmax,
                    name='Maximum_Speed_constraint')

# Print the optimization model info
qp.prettyprint()

"Problem name: \n\nMinimize\n  x_0 + x_1 + x_2 - 0.05*y_0 - 0.05*y_1 - 0.05*y_2\n\nSubject to\n  Linear constraints (14)\n    -x_0 + z_0 <= 0  'z_u_d_0'\n    -x_1 + z_1 <= 0  'z_u_d_1'\n    -x_2 + z_2 <= 0  'z_u_d_2'\n    -y_0 + z_0 <= 0  'z_p_d_0'\n    -y_1 + z_1 <= 0  'z_p_d_1'\n    -y_2 + z_2 <= 0  'z_p_d_2'\n    -x_0 - y_0 + z_0 >= -1  'z_u_p_d_0'\n    -x_1 - y_1 + z_1 >= -1  'z_u_p_d_1'\n    -x_2 - y_2 + z_2 >= -1  'z_u_p_d_2'\n    z_0 + z_1 + z_2 == 0  'No_simultaneous_braking_or_acceleration_constraint'\n    3*x_0 + 2*x_1 + x_2 - 3*y_0 - 2*y_1 - y_2 <= 2  'Max_Distance_constraint'\n    3*x_0 + 2*x_1 + x_2 - 3*y_0 - 2*y_1 - y_2 >= 0  'Min_Distance_constraint'\n    -x_0 - x_1 - x_2 + y_0 + y_1 + y_2 == 0  'Net_Zero_constraint'\n    x_0 + x_1 + x_2 - y_0 - y_1 - y_2 <= 1  'Maximum_Speed_constraint'\n\n  Binary variables (9)\n    x_0 y_0 z_0 x_1 y_1 z_1 x_2 y_2 z_2\n"

In [16]:
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)

: 

In [ ]:
simulator = AerSimulator(method='matrix_product_state',device='GPU')
# Configuring solver
qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA(maxiter=1000))
meo = MinimumEigenOptimizer(qaoa)
# Solving
result_qaoa = meo.solve(qubo)
print(result_qaoa.x)
print(result_qaoa.fval)

/tmp/ipykernel_7932/2819726164.py:3: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA(maxiter=1000))
